In [1]:
import os

from ray import tune
from ray.air.checkpoint import Checkpoint

from PBTTrainable import RayModel
from functionapi import train_net

from analysis import cal_test_accuracy

import torch
import yaml

/home/janis/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-19 16:25:46,120	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-19 16:25:46,343	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    def __getattr__(self, attr):
        if attr.startswith('__'):
            raise AttributeError("Should not start with __")
        return self.get(attr)
    #__getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


args = dotdict()   

experiment_path = "/home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/hapt/01"

#args.trainable_api = 'function'
args.trainable_api = 'class'
args.use_vali_keys = True
args.data_name                        =  "hapt"
args.filter_scaling_factor = 0.25
args.model_type            = "deepconvlstm"
args.model_config = os.path.abspath('configs/model.yaml')


args.seed = 1

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

args.batch_size       = 256                                                    
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90        

args.difference       = False 
args.filtering        = False
args.magnitude        = False
args.weighted_sampler = False

args.pos_select       = None
args.sensor_select    = None

args.representation_type = "time"
args.exp_mode            = "Given"

args.criterion               = "CrossEntropy"


args.freq_save_path   = os.path.abspath("ISWC2022LearnableFilter/Freq_data")
args.window_save_path = os.path.abspath("ISWC2022LearnableFilter/Sliding_window")
args.root_path = os.path.abspath("datasets")

config_file = open('configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"]

if args.difference:
    args.c_in  = args.c_in * 2
if  args.filtering :
    for col in config["sensors"]:
        if "acc" in col:
            args.c_in = args.c_in+1

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1
    
if args.trainable_api=='class':
    trainable = RayModel
else:
    trainable = train_net

In [3]:
restored_tuner = tune.Tuner.restore(experiment_path,trainable=trainable)
results_grid = restored_tuner.get_results()
best_result = results_grid.get_best_result(mode="max", metric="mean_accuracy")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = best_result.checkpoint
if args.trainable_api=='function':
    dict = checkpoint.to_dict()
elif args.trainable_api=='class':
    dict = torch.load(checkpoint._local_path+'/model.pht')
else:
    raise AttributeError()

total_loss,  acc, f_w,  f_macro, f_micro = cal_test_accuracy(args, device, dict)
text = 'Best Trial Test Performance: Total_Loss {} Acc {}  F_w {} F_Macro {} F_Micro {}'.format(total_loss,  acc, f_w,  f_macro, f_micro)
print(text)


2023-08-19 16:25:47,508	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.


Build the DeepConvLSTM model!
 ----------------------- load all the data -------------------
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
[-] Target sampling weights:  [0.00075586 0.00080645 0.00086806 0.00074184 0.00067843 0.00067889
 0.00917431 0.01123596 0.00746269 0.008      0.00595238 0.00909091]
Test data number :  12109
Best Trial Test Performance: Total_Loss 0.25935030854842545 Acc 0.9226195391857296  F_w 0.9215738316339621 F_Macro 0.8082929842791913 F_Micro 0.9226195391857297


In [4]:
best_result.metrics_dataframe

,mean_accuracy,total_loss,random_augmentation_prob,mixup_probability,random_aug_first,done,training_iteration,trial_id,date,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,iterations_since_restore
0,0.471851,1.743721,0.5,0.5,True,False,1,c326c_00000,2023-08-18_22-57-08,1692392228,10.556894,10.556894,173804,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,10.556894,1
1,0.542078,1.284982,0.5,0.5,True,False,2,c326c_00000,2023-08-18_22-57-13,1692392233,5.846180,16.403074,173804,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,16.403074,2
2,0.674985,1.045264,0.5,0.5,True,False,3,c326c_00000,2023-08-18_22-57-19,1692392239,5.857440,22.260514,173804,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,22.260514,3
3,0.759141,0.886043,0.5,0.5,True,False,4,c326c_00000,2023-08-18_22-57-25,1692392245,5.821543,28.082057,173804,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,28.082057,4
4,0.766106,0.801978,0.5,0.5,True,False,5,c326c_00000,2023-08-18_22-57-31,1692392251,5.657537,33.739594,173804,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,33.739594,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.888566,0.342074,0.4,0.2,False,False,96,c326c_00000,2023-08-18_23-29-29,1692394169,3.608521,445.095359,181435,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,3.608521,1
96,0.888566,0.342074,0.4,0.2,False,False,97,c326c_00000,2023-08-18_23-29-33,1692394173,3.698773,448.794133,181435,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,7.307295,2
97,0.888566,0.342074,0.4,0.2,False,False,98,c326c_00000,2023-08-18_23-29-36,1692394176,3.732877,452.527010,181435,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,11.040172,3
98,0.888566,0.342074,0.4,0.2,False,False,99,c326c_00000,2023-08-18_23-29-40,1692394180,3.956904,456.483914,181435,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,14.997076,4
